In [1]:
import pandas as pd
import ast
import json
import base64
import numpy as np
import os
import cv2, random
np.set_printoptions(suppress=True, precision=4)
from scipy import spatial
import numpy as np
import yaml
import os.path as op
from tqdm import tqdm
import nltk
from nltk.corpus import wordnet
tqdm.pandas()

### Features Detection

In [2]:
sg_tsv = './features_detection/training/validation.feature.tsv'
df_feature_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_detection = df_feature_detection.rename(columns={0: "id", 1: "features"})
df_feature_detection

,id,features
0,1018148011,{'features': 'AlyjPQAAAAAAAAAAAAAAALdt2T0AAAAA...
1,1029450589,{'features': 'y/T+PsTyTD8AAAAAxmYSPqyrsj4AAAAA...
2,1029737941,{'features': 'a+xjPdZPMUEAAAAASKzVPx/QGUEAAAAA...
3,103205630,{'features': 'WCzwPgAAAAAAAAAAAAAAAHzvrD4AAAAA...
4,10350842,{'features': 'U8KtPgAAAAAAAAAAudo3PubILz8AAAAA...
...,...,...
1009,86120374,{'features': 'ZPrGPQAAAAAAAAAASnvcP5qNZ0EfyqA7...
1010,86131989,{'features': 'mdCSPwAAAAAAAAAA8EE4PbGDmEAAAAAA...
1011,91875542,{'features': 'k/rdPQAAAAAAAAAAAAAAAHzfjD0AAAAA...
1012,981727078,{'features': 'AAAAAAAAAAAAAAAAAAAAAH3AYzz4gis9...


In [3]:
sg_tsv = './features_detection/training/validation2.label.tsv'
df_label_detection = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_detection=df_label_detection.rename(columns={0: "id", 1: "label"})
df_label_detection

,id,label
0,1018148011,"[{'class': 'Sky', 'conf': 0.9436017274856567, ..."
1,1029450589,"[{'class': 'Wall', 'conf': 0.9430888891220093,..."
2,1029737941,"[{'class': 'Headphone', 'conf': 0.966325402259..."
3,103205630,"[{'class': 'Bottle', 'conf': 0.785697400569915..."
4,10350842,"[{'class': 'Sky', 'conf': 0.9426814317703247, ..."
...,...,...
1009,86120374,"[{'class': 'Hair', 'conf': 0.9508698582649231,..."
1010,86131989,"[{'class': 'Shirt', 'conf': 0.8690727949142456..."
1011,91875542,"[{'class': 'Glasses', 'conf': 0.94794499874114..."
1012,981727078,"[{'class': 'Sky', 'conf': 0.9017960429191589, ..."


In [4]:
full_detection = pd.merge(df_feature_detection, df_label_detection, on="id")
full_detection

,id,features,label
0,1018148011,{'features': 'AlyjPQAAAAAAAAAAAAAAALdt2T0AAAAA...,"[{'class': 'Sky', 'conf': 0.9436017274856567, ..."
1,1029450589,{'features': 'y/T+PsTyTD8AAAAAxmYSPqyrsj4AAAAA...,"[{'class': 'Wall', 'conf': 0.9430888891220093,..."
2,1029737941,{'features': 'a+xjPdZPMUEAAAAASKzVPx/QGUEAAAAA...,"[{'class': 'Headphone', 'conf': 0.966325402259..."
3,103205630,{'features': 'WCzwPgAAAAAAAAAAAAAAAHzvrD4AAAAA...,"[{'class': 'Bottle', 'conf': 0.785697400569915..."
4,10350842,{'features': 'U8KtPgAAAAAAAAAAudo3PubILz8AAAAA...,"[{'class': 'Sky', 'conf': 0.9426814317703247, ..."
...,...,...,...
1009,86120374,{'features': 'ZPrGPQAAAAAAAAAASnvcP5qNZ0EfyqA7...,"[{'class': 'Hair', 'conf': 0.9508698582649231,..."
1010,86131989,{'features': 'mdCSPwAAAAAAAAAA8EE4PbGDmEAAAAAA...,"[{'class': 'Shirt', 'conf': 0.8690727949142456..."
1011,91875542,{'features': 'k/rdPQAAAAAAAAAAAAAAAHzfjD0AAAAA...,"[{'class': 'Glasses', 'conf': 0.94794499874114..."
1012,981727078,{'features': 'AAAAAAAAAAAAAAAAAAAAAH3AYzz4gis9...,"[{'class': 'Sky', 'conf': 0.9017960429191589, ..."


In [5]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_detection.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1014it [00:04, 219.38it/s]


### Features Segmentation

In [6]:
sg_tsv = './features_test1/training/val.feature.tsv'
df_feature_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature_segmentation = df_feature_segmentation.rename(columns={0: "id", 1: "features"})
df_feature_segmentation

,id,features
0,1018148011,{'features': 'cGlYQIDImT92q/g+AAAAABiRbEAAAAAA...
1,1029450589,{'features': 'gq/4QCXKgEHrTThA9AsYQE5O2ED46AhA...
2,1029737941,{'features': 'DGcgQcgjw0AoKupAyl+pQFG9JEEAAAAA...
3,103205630,{'features': 'UgJZQf+q4T9LzTFAG2mFQG2ynkAAAAAA...
4,10350842,{'features': 'PwGfQQAAAADDeSJAhJ0WQZboTz8oBPhA...
...,...,...
1009,86120374,{'features': 'AAAAAAAAAAAAAAAA+C5ZPgAAAAASarlA...
1010,86131989,{'features': '8jCEQABCxj+8tXw/NyoQQWanZT9qnYtB...
1011,91875542,{'features': 'AAAAAKQ9C0AAAAAAupY3QIHlGkGckq0/...
1012,981727078,{'features': 'CtMZQLFWnEHs45Q/5DnaP4V9V0BDH8FA...


In [7]:
sg_tsv = './features_test1/training/val.label.tsv'
df_label_segmentation = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label_segmentation = df_label_segmentation.rename(columns={0: "id", 1: "label"})
df_label_segmentation

,id,label
0,1018148011,"[{'class': 'Truck', 'conf': 0.7444072961807251..."
1,1029450589,"[{'class': 'Sofa', 'conf': 0.9327452182769775,..."
2,1029737941,"[{'class': 'Earphone', 'conf': 0.8826538324356..."
3,103205630,"[{'class': 'Tarp', 'conf': 0.7222883701324463,..."
4,10350842,"[{'class': 'Life jacket', 'conf': 0.6269076466..."
...,...,...
1009,86120374,"[{'class': 'Button', 'conf': 0.965646862983703..."
1010,86131989,"[{'class': 'Water bottle', 'conf': 0.978633344..."
1011,91875542,"[{'class': 'Hinge', 'conf': 0.9092732071876526..."
1012,981727078,"[{'class': 'Pavement', 'conf': 0.9030830860137..."


In [8]:
full_segmentation = pd.merge(df_feature_segmentation, df_label_segmentation, on="id")
full_segmentation

,id,features,label
0,1018148011,{'features': 'cGlYQIDImT92q/g+AAAAABiRbEAAAAAA...,"[{'class': 'Truck', 'conf': 0.7444072961807251..."
1,1029450589,{'features': 'gq/4QCXKgEHrTThA9AsYQE5O2ED46AhA...,"[{'class': 'Sofa', 'conf': 0.9327452182769775,..."
2,1029737941,{'features': 'DGcgQcgjw0AoKupAyl+pQFG9JEEAAAAA...,"[{'class': 'Earphone', 'conf': 0.8826538324356..."
3,103205630,{'features': 'UgJZQf+q4T9LzTFAG2mFQG2ynkAAAAAA...,"[{'class': 'Tarp', 'conf': 0.7222883701324463,..."
4,10350842,{'features': 'PwGfQQAAAADDeSJAhJ0WQZboTz8oBPhA...,"[{'class': 'Life jacket', 'conf': 0.6269076466..."
...,...,...,...
1009,86120374,{'features': 'AAAAAAAAAAAAAAAA+C5ZPgAAAAASarlA...,"[{'class': 'Button', 'conf': 0.965646862983703..."
1010,86131989,{'features': '8jCEQABCxj+8tXw/NyoQQWanZT9qnYtB...,"[{'class': 'Water bottle', 'conf': 0.978633344..."
1011,91875542,{'features': 'AAAAAKQ9C0AAAAAAupY3QIHlGkGckq0/...,"[{'class': 'Hinge', 'conf': 0.9092732071876526..."
1012,981727078,{'features': 'CtMZQLFWnEHs45Q/5DnaP4V9V0BDH8FA...,"[{'class': 'Pavement', 'conf': 0.9030830860137..."


In [9]:
from tqdm import tqdm
bbox_id = 0
for index, row in tqdm(full_segmentation.iterrows()):
    list_features = np.split(np.frombuffer(base64.b64decode(row["features"]["features"]), np.float32), row["features"]["num_boxes"])
    for i in range(row["features"]["num_boxes"]):
        row["label"][i]["feature"] = list_features[i]
        row["label"][i]["bbox_id"] = bbox_id
        bbox_id+=1

1014it [00:01, 597.88it/s]


In [10]:
import en_core_web_md
nlp = en_core_web_md.load()

In [11]:
bbox_similarity = 0.997

for index, row in tqdm(full_segmentation.iterrows()):
    for elem in row["label"]:
        similar_region = {"class": "", "score": 0, "feature": [], "conf": 0}
        similar_regions = {}
        similar_regions_features = {}
        similar_features = {"class": "", "score": 0, "feature": [], "conf": 0}
        for elem2 in full_detection.iloc[index]["label"]:

            if 1 - spatial.distance.cosine(elem["rect"], elem2["rect"]) > bbox_similarity:
                #print("similar region class:", elem2["class"])
                similarity = 1 - spatial.distance.cosine(elem["rect"], elem2["rect"])
                sim_features = 1 - spatial.distance.cosine(elem["feature"], elem2["feature"])
                if (sim_features > similar_features["score"]): # 3
                    similar_features["score"] = sim_features
                    similar_features["class"] = elem2["class"]
                    similar_features["feature"] = elem2["feature"]
                    similar_features["conf"] = elem2["conf"]
                if (similarity > similar_region["score"]): # 1
                    similar_region["score"] = similarity
                    similar_region["class"] = elem2["class"]
                    similar_region["feature"] = elem2["feature"]
                    similar_region["conf"] = elem2["conf"]
                if elem2["class"] in similar_regions: # 2
                    similar_regions[elem2["class"]] += 1
                    if similarity > similar_regions_features[elem2["class"]]["score"]: 
                        similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
                else:
                    similar_regions[elem2["class"]] = 0
                    similar_regions_features[elem2["class"]] = {"score": similarity, "feature": elem2["feature"], "conf": elem2["conf"]}
        if similar_region["class"] != "":
            #print("\n======================================================================")
            final_class = elem["class"]
            elem["class"] = elem["class"].lower()
            #print("Classe vecchia: ", elem["class"])
            classe = max(similar_regions, key=similar_regions.get)
            #print("Classe nuova metodo 1: ", classe)
            sim1 = 1 - spatial.distance.cosine(elem["feature"], similar_regions_features[classe]["feature"])
            tokens = nlp(elem["class"].lower() + " " + classe.lower())
            token1, token2 = tokens[0], tokens[1]
            similarity_seg_1 = token1.similarity(token2)
            #print("Classe nuova metodo 2: ", similar_region["class"])
            sim2 = 1 - spatial.distance.cosine(elem["feature"], similar_region["feature"])
            tokens = nlp(elem["class"].lower() + " " + similar_region["class"].lower())
            token1, token2 = tokens[0], tokens[1]
            similarity_seg_2 = token1.similarity(token2)
            #print("Classe nuova metodo 3: ", similar_features["class"])
            tokens = nlp(elem["class"].lower() + " " + similar_features["class"].lower())
            token1, token2 = tokens[0], tokens[1]
            similarity_seg_3 = token1.similarity(token2)

            peso_feat = 0.6
            peso_word = 0.3
            peso_conf = 0.1

            score_1_sim_feat = similarity_seg_1
            score_1_sim_word = sim1
            score_1_conf = similar_regions_features[classe]["conf"]
            score_1_tot = (score_1_sim_feat*peso_feat + score_1_sim_word*peso_word + score_1_conf*peso_conf)

            score_2_sim_feat = similarity_seg_2
            score_2_sim_word = sim2
            score_2_conf = similar_region["conf"]
            score_2_tot = (score_2_sim_feat*peso_feat + score_2_sim_word*peso_word + score_2_conf*peso_conf)

            score_3_sim_feat = similarity_seg_3
            score_3_sim_word = similar_features["score"]
            score_3_conf = similar_features["conf"]
            score_3_tot = (score_3_sim_feat*peso_feat + score_3_sim_word*peso_word + score_3_conf*peso_conf)

            
            if score_1_tot >= score_2_tot and score_1_tot >= score_3_tot:
                final_class = classe
            elif score_2_tot >= score_1_tot and score_2_tot >= score_3_tot:
                final_class = similar_region["class"]
            elif score_2_tot >= score_1_tot and score_3_tot >= score_2_tot:
                final_class = similar_features["class"]
            elem["class"] = final_class
            #print("classe finale: ", elem["class"])
            #print("======================================================================\n")
            
            
            #img = str(full_detection.iloc[index]["id"])+".jpg"
            #im_cv2 = cv2.imread("../datasets/flickr30k_images/validation/"+img)
            #x,y,w,h = tuple(elem["rect"])
            #ROI = im_cv2[int(y):int(h), int(x):int(w)]
            #cv2.imshow(str(elem["class"]),ROI)
            #cv2.waitKey(0) 
            #cv2.destroyAllWindows()

7it [00:02,  3.89it/s]C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_13184/752841131.py:49: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_seg_1 = token1.similarity(token2)
C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_13184/752841131.py:54: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_seg_2 = token1.similarity(token2)
C:\Users\GABRIE~1.FER\AppData\Local\Temp/ipykernel_13184/752841131.py:58: UserWarning: [W008] Evaluating Token.similarity based on empty vectors.
  similarity_seg_3 = token1.similarity(token2)
1014it [05:57,  2.83it/s]


### Union

In [12]:
full_not_present = []
for i in tqdm(range(len(full_detection))):    
    not_present = []
    for row_dect in full_detection.iloc[i]["label"]:
        check = False
        for row_seg in full_segmentation.iloc[i]["label"]:
            if 1 - spatial.distance.cosine(row_seg["rect"], row_dect["rect"]) > 0.985:
                check = True
                break
        if not check:
            not_present.append(row_dect)
    full_not_present.append(not_present)

100%|██████████████████████████████████████████████████████████████████████████████| 1014/1014 [00:32<00:00, 30.98it/s]


In [13]:
for elem in full_detection["id"].values == full_segmentation["id"].values:
    if not elem: 
        print("noooo")

In [14]:
full_segmentation["not_present"] = full_not_present

In [15]:
full_segmentation["union"] = full_segmentation["label"] + full_segmentation["not_present"]
full_segmentation

,id,features,label,not_present,union
0,1018148011,{'features': 'cGlYQIDImT92q/g+AAAAABiRbEAAAAAA...,"[{'class': 'Truck', 'conf': 0.7444072961807251...","[{'class': 'Pile', 'conf': 0.6064555048942566,...","[{'class': 'Truck', 'conf': 0.7444072961807251..."
1,1029450589,{'features': 'gq/4QCXKgEHrTThA9AsYQE5O2ED46AhA...,"[{'class': 'Bed', 'conf': 0.9327452182769775, ...","[{'class': 'Wall', 'conf': 0.9430888891220093,...","[{'class': 'Bed', 'conf': 0.9327452182769775, ..."
2,1029737941,{'features': 'DGcgQcgjw0AoKupAyl+pQFG9JEEAAAAA...,"[{'class': 'Headphone', 'conf': 0.882653832435...","[{'class': 'Shirt', 'conf': 0.8278228044509888...","[{'class': 'Headphone', 'conf': 0.882653832435..."
3,103205630,{'features': 'UgJZQf+q4T9LzTFAG2mFQG2ynkAAAAAA...,"[{'class': 'Cloth', 'conf': 0.7222883701324463...","[{'class': 'Bottle', 'conf': 0.785697400569915...","[{'class': 'Cloth', 'conf': 0.7222883701324463..."
4,10350842,{'features': 'PwGfQQAAAADDeSJAhJ0WQZboTz8oBPhA...,"[{'class': 'Vest', 'conf': 0.6269076466560364,...","[{'class': 'Sky', 'conf': 0.9426814317703247, ...","[{'class': 'Vest', 'conf': 0.6269076466560364,..."
...,...,...,...,...,...
1009,86120374,{'features': 'AAAAAAAAAAAAAAAA+C5ZPgAAAAASarlA...,"[{'class': 'Button', 'conf': 0.965646862983703...","[{'class': 'Hair', 'conf': 0.9508698582649231,...","[{'class': 'Button', 'conf': 0.965646862983703..."
1010,86131989,{'features': '8jCEQABCxj+8tXw/NyoQQWanZT9qnYtB...,"[{'class': 'Belt', 'conf': 0.9786333441734314,...","[{'class': 'Bus', 'conf': 0.5938499569892883, ...","[{'class': 'Belt', 'conf': 0.9786333441734314,..."
1011,91875542,{'features': 'AAAAAKQ9C0AAAAAAupY3QIHlGkGckq0/...,"[{'class': 'Hinge', 'conf': 0.9092732071876526...","[{'class': 'Hand', 'conf': 0.810876727104187, ...","[{'class': 'Hinge', 'conf': 0.9092732071876526..."
1012,981727078,{'features': 'CtMZQLFWnEHs45Q/5DnaP4V9V0BDH8FA...,"[{'class': 'Sidewalk', 'conf': 0.9030830860137...","[{'class': 'Sky', 'conf': 0.9017960429191589, ...","[{'class': 'Sidewalk', 'conf': 0.9030830860137..."


In [16]:
for index, row in tqdm(full_segmentation.iterrows()):
    if(len(row["label"]) + len(row["not_present"]) != len(row["union"])):
        print("!!!!!!!!!!!!!!!")

1014it [00:00, 9218.63it/s]


In [17]:
def generate_features(x):
    idx, data, num_boxes = x["id"],x["union"],len(x["union"])
    features_arr = []
    for i in range(num_boxes):
        features = data[i]['feature']
        features_arr.append(features.astype(np.float32))
    
    features = np.vstack(tuple(features_arr))
    features = base64.b64encode(features).decode("utf-8")
    return {"features":features, "num_boxes":num_boxes}

In [18]:
def generate_labels(x):
    data = x["union"]
    res = [{"class":el['class'].capitalize(),"conf":el['conf'], "rect": el['rect']} for el in data] 
    return res

In [19]:
full_segmentation['feature_union'] = full_segmentation.progress_apply(generate_features,axis=1)
full_segmentation['feature_union'] = full_segmentation['feature_union'].progress_apply(json.dumps)

full_segmentation['label_union'] = full_segmentation.progress_apply(generate_labels,axis=1)
full_segmentation['label_union'] = full_segmentation['label_union'].progress_apply(json.dumps)

100%|████████████████████████████████████████████████████████████████████████████| 1014/1014 [00:00<00:00, 6673.81it/s]


In [20]:
from features_detection.maskrcnn_benchmark.structures.tsv_file_ops import tsv_reader, tsv_writer

OUTPUT_DIR = './training_union_class/'
LABEL_FILE = os.path.join(OUTPUT_DIR,'val.label.tsv')
FEATURE_FILE = os.path.join(OUTPUT_DIR,'val.feature.tsv')
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)
    print(f"path to {OUTPUT_DIR} created")

In [21]:
tsv_writer(full_segmentation[["id", "feature_union"]].values.tolist(), FEATURE_FILE)

In [22]:
tsv_writer(full_segmentation[["id", "label_union"]].values.tolist(),LABEL_FILE)

In [24]:
sg_tsv = './training_union_class/val.feature.tsv'
df_feature = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_feature = df_feature.rename(columns={0: "id", 1: "features"})
df_feature

,id,features
0,1018148011,{'features': 'cGlYQIDImT92q/g+AAAAABiRbEAAAAAA...
1,1029450589,{'features': 'gq/4QCXKgEHrTThA9AsYQE5O2ED46AhA...
2,1029737941,{'features': 'DGcgQcgjw0AoKupAyl+pQFG9JEEAAAAA...
3,103205630,{'features': 'UgJZQf+q4T9LzTFAG2mFQG2ynkAAAAAA...
4,10350842,{'features': 'PwGfQQAAAADDeSJAhJ0WQZboTz8oBPhA...
...,...,...
1009,86120374,{'features': 'AAAAAAAAAAAAAAAA+C5ZPgAAAAASarlA...
1010,86131989,{'features': '8jCEQABCxj+8tXw/NyoQQWanZT9qnYtB...
1011,91875542,{'features': 'AAAAAKQ9C0AAAAAAupY3QIHlGkGckq0/...
1012,981727078,{'features': 'CtMZQLFWnEHs45Q/5DnaP4V9V0BDH8FA...


In [23]:
sg_tsv = './training_union_class/val.label.tsv'
df_label = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_label = df_label.rename(columns={0: "id", 1: "label"})
df_label

,id,label
0,1018148011,"[{'class': 'Truck', 'conf': 0.7444072961807251..."
1,1029450589,"[{'class': 'Bed', 'conf': 0.9327452182769775, ..."
2,1029737941,"[{'class': 'Headphone', 'conf': 0.882653832435..."
3,103205630,"[{'class': 'Cloth', 'conf': 0.7222883701324463..."
4,10350842,"[{'class': 'Vest', 'conf': 0.6269076466560364,..."
...,...,...
1009,86120374,"[{'class': 'Button', 'conf': 0.965646862983703..."
1010,86131989,"[{'class': 'Belt', 'conf': 0.9786333441734314,..."
1011,91875542,"[{'class': 'Hinge', 'conf': 0.9092732071876526..."
1012,981727078,"[{'class': 'Sidewalk', 'conf': 0.9030830860137..."


In [27]:
sg_tsv = './training_union/val.hw.tsv'
df_hw = pd.read_csv(sg_tsv,sep='\t',header = None, converters={1:json.loads})
df_hw = df_hw.rename(columns={0: "id", 1: "dim"})
df_hw

,id,dim
0,1018148011,"[{'height': 333, 'width': 500}]"
1,1029450589,"[{'height': 334, 'width': 500}]"
2,1029737941,"[{'height': 500, 'width': 375}]"
3,103205630,"[{'height': 215, 'width': 500}]"
4,10350842,"[{'height': 375, 'width': 500}]"
...,...,...
1009,86120374,"[{'height': 500, 'width': 464}]"
1010,86131989,"[{'height': 464, 'width': 500}]"
1011,91875542,"[{'height': 375, 'width': 500}]"
1012,981727078,"[{'height': 500, 'width': 375}]"


In [28]:
np.all(df_hw["id"].values == df_feature["id"].values)

True

In [29]:
np.all(df_hw["id"].values == df_label["id"].values)

True